In [1]:
#Import the necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
#Make the discriminator
#Here img_data = 28*28 = 784
#Here LeakyReLU is used, SeLU can also be used instead

class Discriminator(nn.Module):
    def __init__(self, img_data):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_data, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)


In [3]:
#Make the generator
#Here the inputs are the random noise, denoted by z_dim and img_dim = 28*28 = 784
#Here LeakyReLU is used, SeLU can also be used instead

#The last layer is Tanh, instead of sigmoid because Tanh has a even distribution across zero and has
#better gradients while optimizing than sigmoid. The pixel values are dealt later in the subsequent code.

class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)

In [4]:
#hyperparameters
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((25, z_dim)).to(device)  #at every epoch we generate 25 images

transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

In [5]:
#MNIST Dataset
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11503214.41it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 269113.88it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3192182.17it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2558835.29it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [6]:
#function to generate and save images at every epoch

def generate_and_save_images(gen, epoch):
    with torch.no_grad():
        fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
        fake = fake * 0.5 + 0.5  # rescale from [-1, 1] to [0, 1]
        plt.figure(figsize=(5, 5))
        for i in range(25):
            plt.subplot(5, 5, i + 1)
            plt.imshow(fake[i][0].cpu(), cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'generated_images_epoch_{epoch+1}.png')
        plt.close()

In [7]:
#Training loop

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    running_lossD = 0.0
    running_lossG = 0.0
    total_batches = len(loader)
    
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        running_lossD += lossD.item()
        running_lossG += lossG.item()

    avg_lossD = running_lossD / total_batches
    avg_lossG = running_lossG / total_batches
    tqdm.write(f"Epoch [{epoch+1}/{num_epochs}] Loss D: {avg_lossD:.4f}, Loss G: {avg_lossG:.4f}")
    
    generate_and_save_images(gen, epoch)

print("training finished!")


Epochs:   0%|          | 0/50 [00:20<?, ?it/s]

Epoch [1/50] Loss D: 0.4090, Loss G: 1.3100


Epochs:   2%|▏         | 1/50 [00:40<17:05, 20.93s/it]

Epoch [2/50] Loss D: 0.5662, Loss G: 1.0510


Epochs:   4%|▍         | 2/50 [01:00<16:19, 20.41s/it]

...

Epoch [49/50] Loss D: 0.6304, Loss G: 1.0843


Epochs:  98%|█████████▊| 49/50 [16:51<00:20, 20.13s/it]

Epoch [50/50] Loss D: 0.5975, Loss G: 1.1142


Epochs: 100%|██████████| 50/50 [16:52<00:00, 20.25s/it]

training finished!


In [8]:
torch.save(gen.state_dict(), "generator.pth")
torch.save(disc.state_dict(), "discriminator.pth")